## 多进程

Linux操作系统提供了一个__fork()__系统调用，非常特殊，普通的函数调用，调用一次返回一次，但__fork()__调用一次，返回两次，因为操作系统自动把当前进程（称为父进程）复制了一份（称为子进程）,然后，分别在父进程和子进程内返回。

子进程永远返回__0__,而父进程返回子进程的__ID__。这样做的理由是,一个父进程可以__fork__出很多子进程，所以，父进程要记下每个子进程的ID，而子进程需要调用__getppid()__就可以拿到父进程的__ID__

In [11]:
import os

print('Process (%s) start...' % os.getpid())

pid = os.fork()
if pid ==0:
    print("I'm child process (%s) and my parent is %s." % (os.getpid(), os.getppid()))
else:
    print("I (%s) just created a child process (%s)." % (os.getpid(), pid))

Process (22370) start...
I (22370) just created a child process (22819).
I'm child process (22819) and my parent is 22370.


- 有了fork调用，一个进程在接到新任务时就可以复制出一个子进程来处理新任务，常见的Apache服务器就是由父进程监听端口，每当有新的http请求时，就fork出子进程来处理新的http请求。

## multiprocessing

In [10]:
from multiprocessing import Process
import os

# 子进程要执行的代码
def run_proc(name):
    print('Run child process %s (%s)...' % (name, os.getpid()))
    
if __name__ == '__main__':
    print("Parent process %s." % os.getpid())
    p = Process(target=run_proc, args=('test',))
    print('Child process will start.')
    p.start()
    p.join()
    print('Child process end.')

Parent process 22370.
Child process will start.
Run child process test (22728)...
Child process end.


创建子进程时，只需要传入一个执行函数和函数的参数，创建一个Process实例，用start()方法启动，这样创建进程比fork()还要简单。

join()方法可以等待子进程结束后再继续往下运行，通常用于进程间的同步。

 ### Pool
 
如果要启动大量的子进程， 可以用进程池的方法批量创建子进程：

In [13]:
from multiprocessing import Pool
import os, time, random

def long_time_task(name):
    print('Run task %s (%s)...' %  (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' % (name, end - start))
    
if __name__ == '__main__':
    print('Parent process %s.' % os.getpid())
    p = Pool(4)
    for i in range(5):
        p.apply_async(long_time_task, args=(i,))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')

Parent process 22370.
Run task 0 (22872)...
Run task 1 (22873)...
Run task 2 (22874)...
Run task 3 (22875)...
Waiting for all subprocesses done...
Task 2 runs 1.13 seconds.
Run task 4 (22874)...
Task 3 runs 1.30 seconds.
Task 1 runs 1.92 seconds.
Task 0 runs 2.14 seconds.
Task 4 runs 2.54 seconds.
All subprocesses done.


## 子进程

很多时候， 子进程并不是自身， 而是一个外部进程， 我们创建了子进程后， 还需要控制子进程的输入和输出。

__subprocess__模块可以让我们非常方便地启动一个子进程， 然后控制其输入和输出。

下面的例子演示了如何在python代码运行命令nslookup www.python.org, 这和命令直接运行的效果一样：

In [14]:
import subprocess

print('$ nslookup www.python.org')
r = subprocess.call(['nslookup', 'www.python.org'])
print('Exit code:', r)

$ nslookup www.python.org
Exit code: 0


In [18]:
# 如果子进程还需要输入， 则可以通过communicate()方法输入：
import subprocess

print('$ nslookup')
p = subprocess.Popen(['nslookup'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
output, err = p.communicate(b'set q=mx\npython.org\nexit\n')
print(output.decode('utf-8'))
print('Exit code:', p.returncode)

$ nslookup
Server:		8.8.8.8
Address:	8.8.8.8#53

Non-authoritative answer:
python.org	mail exchanger = 50 mail.python.org.

Authoritative answers can be found from:


Exit code: 0


### 进程间通信
__process__ 之间肯定要通信的， 操作系统提供了许多机制来实现通信。 python 的__multiprocessing__模块包装了底层的机制，提供了__Queue__,__Pipes__等多种方式来交换数据

In [20]:
from multiprocessing import Process, Queue

# 写入数据执行的代码：
def write(q):
    print('Process to write: %s' % os.getpid())
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())
        
# 读取数据进程执行的代码：
def read(q):
    print('Process to read: %s' % os.getpid())
    while True:
        value = q.get(True)
        print('Get %s from queue.' % value)
        
if __name__ == '__main__':
    # 父进程创建Queue，并传给各个子进程
    q = Queue()
    pw = Process(target=write, args=(q,))
    pr = Process(target=read, args=(q,))
    # 启动子进程qw， 写入：
    pw.start()
    # 启动子进程qr, 读取：
    pr.start()
    # 等待pw 结束
    pw.join()
    # pr进程里是死循环，无法等待其结束，只能强行终止
    pr.terminate()

Process to write: 23799
Process to read: 23802
Put A to queue...
Get A from queue.
Put B to queue...
Get B from queue.
Put C to queue...
Get C from queue.


小结

在Unix/Linux下，可以使用fork()调用实现多进程。

要实现跨平台的多进程，可以使用multiprocessing模块。

进程间通信是通过Queue、Pipes等实现的。